In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
load_dotenv()
import os

key = os.getenv("Groq_api_key")

model_client =  OpenAIChatCompletionClient(
    base_url="https://api.groq.com/openai/v1",
    model="llama-3.3-70b-versatile",
    api_key = key,
    model_info={
        "family":'llama',
        "vision" :False,
        "function_calling":True,
        "json_output": True
    }
)

c:\Users\shubu\Desktop\AutoGen Hardcode\venv\lib\site-packages\autogen_ext\models\openai\_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


## Why use a single Agent team?

1. Consistency - termination conditions, logging, observing
2. Scalability - we can add more agents as our use case expands without rewriting logic. 
3. Control - Features like observers or cancellation work seamlessly.

In [2]:
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
poet = AssistantAgent(
    name = 'solo_poet',
    model_client=model_client,
    system_message="You are a poet. Write a short poem on Moon and end with 'MOONLIGHT FADES'. "
)

termination = TextMentionTermination('MOONLIGHT FADES')

team = RoundRobinGroupChat(
    participants=[poet],
    termination_condition=termination
)

In [3]:
async def test_single_agent():
    task = TextMessage(content="Write a short poem on in 30 words on Moon",source='user')
    result = await team.run(task=task)
    for message in result.messages:
        print(f"{message.source}: {message.content}")


await test_single_agent()  

user: Write a short poem on in 30 words on Moon
solo_poet: Softly glows the Moon, a silver orb in night, radiant beams illuminating all, serene and peaceful, gentle MOONLIGHT FADES


## Single-Agent Team vs. Plain Single Agent

| Aspect             | Single-Agent Team            | Plain Single Agent          |
|--------------------|------------------------------|-----------------------------|
| **Termination**    | Stops with condition (e.g., "MOONLIGHT FADES") | Runs until task ends        |
| **Observability**  | Supports Console observers   | Manual logging only         |
| **Flexibility**    | Easily scales to multi-agent | Fixed to one agent          |